In [3]:
import pandas as pd
import numpy as np
import sqlite3
import ast

In [4]:
df_events = pd.read_csv(
    r"C:/Users/pc/Python_Projects/final_project/final_data/2023_24hackle/df_hackle_processed_20230718-08.csv",
    dtype={"item_name": str, "osversion": str},  # 문제 있는 컬럼을 문자열(str)로 변환
    low_memory=False  # 메모리 분할 로딩 방지
)


In [5]:
# SQLite 연결
conn = sqlite3.connect("network_analysis.db")
cursor = conn.cursor()

# 유저 정보 데이터 로드
df_users = pd.read_csv(r"C:/Users/pc/Python_Projects/final_project/final_data/hackle/votes_user_total.csv")

# friend_id_list 컬럼을 리스트로 변환
df_users["friend_id_list"] = df_users["friend_id_list"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else [])

# users 테이블 생성
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id INTEGER PRIMARY KEY,
        gender TEXT,
        grade REAL,
        school_id REAL,
        friends_num INTEGER
    )
""")

# 데이터 삽입
for _, row in df_users.iterrows():
    cursor.execute("INSERT INTO users (user_id, gender, grade, school_id, friends_num) VALUES (?, ?, ?, ?, ?)", 
                   (row["user_id"], row["gender"], row["grade"], row["school_id"], row["friends_num"]))

# 친구 관계 테이블 생성
cursor.execute("""
    CREATE TABLE IF NOT EXISTS friendships (
        user_id INTEGER,
        friend_id INTEGER,
        PRIMARY KEY (user_id, friend_id)
    )
""")

# 친구 관계 데이터 삽입
for _, row in df_users.iterrows():
    user_id = row["user_id"]
    for friend_id in row["friend_id_list"]:
        cursor.execute("INSERT OR IGNORE INTO friendships (user_id, friend_id) VALUES (?, ?)", (user_id, friend_id))

# 이벤트 데이터 로드
df_events = pd.read_csv(r"C:/Users/pc/Python_Projects/final_project/final_data/2023_24hackle/df_hackle_processed_20230718-08.csv")

# events 테이블 생성
cursor.execute("""
    CREATE TABLE IF NOT EXISTS events (
        user_id INTEGER,
        event_datetime TEXT,
        event_key TEXT,
        date TEXT
    )
""")

# 데이터 삽입
for _, row in df_events.iterrows():
    cursor.execute("INSERT INTO events (user_id, event_datetime, event_key, date) VALUES (?, ?, ?, ?)", 
                   (row["user_id"], row["event_datetime"], row["event_key"], row["date"]))

# 친구 요청 테이블 생성
df_requests = pd.read_csv(r"C:/Users/pc/Python_Projects/final_project/final_data/pjdata/votes/accounts_friendrequest.csv")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS friend_requests (
        receive_user_id INTEGER,
        send_user_id INTEGER,
        status TEXT,
        created_at TEXT
    )
""")

# 데이터 삽입
for _, row in df_requests.iterrows():
    cursor.execute("INSERT INTO friend_requests (receive_user_id, send_user_id, status, created_at) VALUES (?, ?, ?, ?)", 
                   (row["receive_user_id"], row["send_user_id"], row["status"], row["created_at"]))

conn.commit()
conn.close()


C:\Users\pc\AppData\Local\Temp\ipykernel_14008\4267760669.py:43: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_events = pd.read_csv(r"C:/Users/pc/Python_Projects/final_project/final_data/2023_24hackle/df_hackle_processed_20230718-08.csv")
